In [11]:
nn.BatchNorm2d()

NameError: name 'nn' is not defined

In [1]:
import numpy as np
import time
import os
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import models
from torchsummary import summary
import torch.nn as nn
import torch
from torch.autograd.variable import Variable
from torchvision import datasets, models, transforms
import math
import torch.utils.model_zoo as model_zoo

In [ ]:
r= models.resnet18(pretrained=False)

In [ ]:
r.forward()

In [2]:
model = models.resnet18(pretrained=False)

In [3]:
counter = 0
for child in model.children():
    #print(child)
    print("--", counter,'----------', child)
    counter += 1

-- 0 ---------- Conv2d(13, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
-- 1 ---------- BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
-- 2 ---------- ReLU(inplace)
-- 3 ---------- MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
-- 4 ---------- Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (conv2): Conv2d(6

In [4]:
# Changing last layer
num_final_in = model.fc.in_features

# The final layer of the model is model.fc so we can basically just overwrite it 
# to have the output = number of classes we need. Say, 300 classes.
NUM_CLASSES = 3
model.fc = nn.Linear(num_final_in, NUM_CLASSES)

In [6]:
summary(model, (13,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]          40,768
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [20]:
dummy_image_1 = np.random.rand(224,224,12)
dummy_image_2 = np.random.rand(224,224,12)
dummy_image_3 = np.random.rand(224,224,12)
dummy_image_4 = np.random.rand(224,224,12)
dummy_image_5 = np.random.rand(224,224,12)

#### Defining Loss function, Optimizer etc

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
els = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

####  let’s write a general function to train a model
We will illustrate: 
1. Scheduling the learning rate
2. Saving the best model

In the following, parameter scheduler is an LR scheduler object from torch.optim.lr_scheduler.

In [9]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [35]:
model = train_model(model, criterion, optimizer_ft, els, num_epochs=25)

NameError: name 'train_model' is not defined

In [38]:
partition = dict()

In [39]:
partition['train'] = ['id-1', 'id-2', 'id-3']

In [41]:
partition['validation'] = ['id-4']

In [43]:
labels = dict()

In [44]:
labels['id-1'] = 0
labels['id-2'] = 1
labels['id-3'] = 2
labels['id-4'] = 1

In [10]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch

### Creating Custom Datasets

In [11]:
# Creating custom Dataset classes

class MyDataset(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).float()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
            
        return x, y
    
    def __len__(self):
        return len(self.data)

In [23]:
numpy_data = np.random.randn(100,12,224,224)
numpy_target = np.random.randint(0,3,size=(100))

In [24]:
print(numpy_data.shape)
print(numpy_target.shape)

(100, 12, 224, 224)
(100,)


In [26]:
numpy_target

array([2, 1, 1, 0, 1, 1, 2, 2, 0, 2, 1, 0, 0, 2, 1, 0, 0, 2, 1, 1, 2, 0,
       2, 2, 0, 0, 1, 2, 2, 2, 0, 2, 1, 1, 1, 0, 0, 1, 2, 0, 1, 2, 1, 2,
       1, 1, 2, 1, 1, 0, 2, 1, 2, 1, 2, 2, 0, 0, 2, 0, 0, 1, 1, 1, 2, 0,
       0, 1, 2, 1, 0, 1, 1, 0, 2, 0, 1, 2, 1, 2, 2, 2, 2, 0, 1, 0, 1, 1,
       2, 0, 0, 1, 0, 1, 2, 1, 2, 1, 0, 2])

In [14]:
numpy_target

array([2, 1, 2, 0, 0, 0, 1, 2, 2, 1, 2, 0, 1, 2, 1, 1, 1, 2, 0, 1, 2, 2,
       1, 2, 1, 1, 1, 0, 0, 2, 1, 2, 0, 0, 0, 1, 2, 0, 1, 0, 2, 0, 1, 1,
       2, 1, 0, 2, 1, 0, 1, 0, 1, 1, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 0,
       2, 2, 2, 2, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0])

In [15]:
dataset = MyDataset(numpy_data, numpy_target)

In [17]:
print(dataset.__len__())
print(dataset.__getitem__(9)[1])
#print(dataset.__getitem__(2)[0])

100
tensor(1)


In [18]:
#loader = DataLoader(dataset, batch_size=10, shuffle=True, num_workers=2, pin_memory=torch.cuda.is_available() )
loader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=0, pin_memory=True)

In [19]:
for batch_idx, (data, target) in enumerate(loader):
    print('Batch idx {}, data shape {}, target shape {}'.format(batch_idx, data.shape, target.shape))

Batch idx 0, data shape torch.Size([16, 12, 224, 224]), target shape torch.Size([16])
Batch idx 1, data shape torch.Size([16, 12, 224, 224]), target shape torch.Size([16])
Batch idx 2, data shape torch.Size([16, 12, 224, 224]), target shape torch.Size([16])
Batch idx 3, data shape torch.Size([16, 12, 224, 224]), target shape torch.Size([16])
Batch idx 4, data shape torch.Size([16, 12, 224, 224]), target shape torch.Size([16])
Batch idx 5, data shape torch.Size([16, 12, 224, 224]), target shape torch.Size([16])
Batch idx 6, data shape torch.Size([4, 12, 224, 224]), target shape torch.Size([4])


In [20]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    
    for epoch in range(num_epochs):
        
        scheduler.step()
        model.train()
        
        for i, (images,labels) in enumerate(loader):
            images = images.to(device)
            labels = labels.to(device)
            
            # Forward Pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            #Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()    
            
            print('Epoch [{}/{}],  Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, 100, loss.item()))

In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [22]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
els = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [23]:
model = train_model(model, criterion, optimizer_ft, els, num_epochs=3)

Epoch [1/3],  Step [1/100], Loss: 1.1542
Epoch [1/3],  Step [2/100], Loss: 1.1598
Epoch [1/3],  Step [3/100], Loss: 1.4662
Epoch [1/3],  Step [4/100], Loss: 1.0993
Epoch [1/3],  Step [5/100], Loss: 1.0935
Epoch [1/3],  Step [6/100], Loss: 1.1134
Epoch [1/3],  Step [7/100], Loss: 1.1910
Epoch [2/3],  Step [1/100], Loss: 0.9405
Epoch [2/3],  Step [2/100], Loss: 0.9349
Epoch [2/3],  Step [3/100], Loss: 1.0179
Epoch [2/3],  Step [4/100], Loss: 1.1639
Epoch [2/3],  Step [5/100], Loss: 1.2163
Epoch [2/3],  Step [6/100], Loss: 0.9425
Epoch [2/3],  Step [7/100], Loss: 0.8593
Epoch [3/3],  Step [1/100], Loss: 0.7845
Epoch [3/3],  Step [2/100], Loss: 0.7759
Epoch [3/3],  Step [3/100], Loss: 0.7827
Epoch [3/3],  Step [4/100], Loss: 0.8040
Epoch [3/3],  Step [5/100], Loss: 0.8175
Epoch [3/3],  Step [6/100], Loss: 0.9009
Epoch [3/3],  Step [7/100], Loss: 0.7338


In [24]:
numpy_data_test = np.random.randn(10,12,224,224)
numpy_target_test = np.random.randint(0,3,size=(10))
dataset = MyDataset(numpy_data_test, numpy_target_test)
loader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0, pin_memory=True)

In [34]:
for batch_idx, (data, target) in enumerate(loader):
    print('Batch idx {}, data shape {}, target shape {}'.format(batch_idx, data.shape, target.shape))

Batch idx 0, data shape torch.Size([4, 12, 224, 224]), target shape torch.Size([4])
Batch idx 1, data shape torch.Size([4, 12, 224, 224]), target shape torch.Size([4])
Batch idx 2, data shape torch.Size([2, 12, 224, 224]), target shape torch.Size([2])


In [35]:
# Testing the model
def eval_model(model,test_loader):
    
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
        print('Test Accuracy on 10 images: {} %'.format(100*correct/total))

In [36]:
eval_model(model, loader)

AttributeError: 'NoneType' object has no attribute 'eval'

## Loading Data from Folder

In [1]:
import rasterio
import os
import numpy as np
import pandas as pd

In [7]:
#Extracting Labels
df=pd.read_csv(r"C:\Users\AJain7\OneDrive - Stryker\Personal\Projects\Satellite Project\Labels\VillageLabels_MSW.csv")
village_code=df["Town/Village"].values
emp_label=df["Village_HHD_Cluster_MSW"].values
actual_labels= [ int(c) for c in emp_label]
s1 = pd.Series(actual_labels,index=list(village_code))

def slice_x(img,resize_dim_x):
    x,y,_ = img.shape
    startx = x//2-(resize_dim_x//2)
    return img[startx:startx+resize_dim_x, :,:]

def slice_y(img,resize_dim_y):
    x,y,_ = img.shape
    starty = y//2-(resize_dim_y//2)
    return img[:,starty:starty+resize_dim_y,:]

def crop_center(img,cropx,cropy): #Function for cropping the image from the center
    y,x = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)
    return img[starty:starty+cropy,startx:startx+cropx]

all_img = []
all_label = np.array([])
resizeDim = 64
nchannels = 3
path = r"C:\Users\AJain7\Downloads\sat_images"
for file in os.listdir(path):
    filename = os.path.join(path,file)
    dataset = rasterio.open(filename)
    village_code = int(file.split('@')[3].split('.')[0])
    label = s1.loc[village_code]
    all_label = np.append(all_label,label)
    
    #X=np.array([]).reshape((0,resizeDim,resizeDim, nchannels))
    band1 = dataset.read(1)
    band2 = dataset.read(2)
    band3 = dataset.read(3)
    band4 = dataset.read(4)
    band5 = dataset.read(5)
    band6 = dataset.read(6)
    band7 = dataset.read(7)
    band8 = dataset.read(8)
    band9 = dataset.read(9)
    band10 = dataset.read(10)
    band11 = dataset.read(11)
    band12 = dataset.read(12)
    band13 = dataset.read(13)
    #cd = np.dstack((band1, band2, band3, band4, band5, band6, band7, band8, band9, band10, band11, band12, band13))
    cd = np.dstack((band1, band2, band3))
    print(cd.shape)
    
    if (cd.shape[0] > resizeDim or cd.shape[1] > resizeDim):
        #print('shape before slicing',cd.shape)
        #print('slicing x')
        if(cd.shape[0] > resizeDim and cd.shape[1] > resizeDim):
            #print('Slicing both together')
            combinedData = slice_x(cd,resize_dim_x=resizeDim)
            combinedData = slice_y(combinedData, resize_dim_y=resizeDim)
            #print('shape after slicing',combinedData.shape)
            #print('FOCUS HERE--------------------------------------------------------------')
        elif(cd.shape[0] > resizeDim and cd.shape[1] <= resizeDim):
            #print('Slicing x')
            combinedData = slice_x(cd,resize_dim_x=resizeDim)
            #print('shape after slicing',combinedData.shape)
            #print('FOCUS HERE--------------------------------------------------------------')
        elif(cd.shape[0] <= resizeDim and cd.shape[1] > resizeDim):
            #print('slicing y')
            combinedData = slice_y(cd, resize_dim_y=resizeDim)
            #print('shape after slicing',combinedData.shape)
            #print('FOCUS HERE--------------------------------------------------------------')
    else:
        combinedData = cd
        #print('Shape without slicing', combinedData.shape)
    
    left = (resizeDim-combinedData.shape[0])//2
    right = resizeDim-combinedData.shape[0] - left
    print(left, right)
    up = (resizeDim-combinedData.shape[1])//2
    down = resizeDim-combinedData.shape[1] - up
    print(up, down)
    
    
    data = np.lib.pad(combinedData, [(left,right),(up,down),(0,0)], 'constant')
    #print('Before Shaping',data.shape)
    data = np.reshape(data,(1,nchannels,resizeDim,resizeDim))
    print(data.shape)
    print('-------------------------------------------------')
    #print('After Shaping',data.shape)
    #print('  ')
    all_img.append(data)
ai = np.vstack(all_img)

(160, 184, 3)
0 0
0 0
(1, 3, 64, 64)
-------------------------------------------------
(112, 169, 3)
0 0
0 0
(1, 3, 64, 64)
-------------------------------------------------
(99, 96, 3)
0 0
0 0
(1, 3, 64, 64)
-------------------------------------------------
(146, 216, 3)
0 0
0 0
(1, 3, 64, 64)
-------------------------------------------------
(103, 98, 3)
0 0
0 0
(1, 3, 64, 64)
-------------------------------------------------
(161, 180, 3)
0 0
0 0
(1, 3, 64, 64)
-------------------------------------------------
(88, 127, 3)
0 0
0 0
(1, 3, 64, 64)
-------------------------------------------------
(81, 82, 3)
0 0
0 0
(1, 3, 64, 64)
-------------------------------------------------
(96, 71, 3)
0 0
0 0
(1, 3, 64, 64)
-------------------------------------------------
(126, 110, 3)
0 0
0 0
(1, 3, 64, 64)
-------------------------------------------------
(98, 148, 3)
0 0
0 0
(1, 3, 64, 64)
-------------------------------------------------
(80, 148, 3)
0 0
0 0
(1, 3, 64, 64)
----------------